In [ ]:
# Step 1: Download and unzip dataset from Roboflow
!curl -L "https://app.roboflow.com/ds/t6xM6xE9bF?key=ym8EYKP9vQ" > roboflow.zip
!unzip roboflow.zip -d dataset && rm roboflow.zip

# Step 2: Import necessary libraries
import os
import torch
import torchvision
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report

# Step 3: Define device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Step 4: Define transformations for training and validation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Step 5: Load dataset from Roboflow structure (assumes folder structure: dataset/train/class_name/)
train_dir = '/content/dataset/train'
val_dir = '/content/dataset/valid'

if not os.path.exists(val_dir):
    val_dir = train_dir  # use train if no validation set provided

train_data = datasets.ImageFolder(train_dir, transform=transform)
val_data = datasets.ImageFolder(val_dir, transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

# Step 6: Load and modify ResNet-50
model = models.resnet50(pretrained=True)
num_classes = len(train_data.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Step 7: Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Step 8: Train the model
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

# Step 9: Evaluate the model
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds, target_names=train_data.classes))


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   904  100   904    0     0   3555      0 --:--:-- --:--:-- --:--:--  3545
100 17.3M  100 17.3M    0     0  14.4M      0  0:00:01  0:00:01 --:--:-- 14.4M
Archive:  roboflow.zip
  inflating: dataset/README.dataset.txt  
  inflating: dataset/README.roboflow.txt  
  inflating: dataset/data.yaml       
   creating: dataset/test/
   creating: dataset/test/images/
 extracting: dataset/test/images/07-is-250-came-to-my-car-saw-a-big-crack-in-the-rear-v0-epzwgdws1iaa1_webp.rf.f8fa8c1e25c320516c8be752fc2c6fbe.jpg  
 extracting: dataset/test/images/073_jpg.rf.9fb7792655a0d1aeb80a19d88d2216c2.jpg  
 extracting: dataset/test/images/077_jpg.rf.27e0ff0931694f718ff9ce851b1a7ae9.jpg  
 extracting: dataset/test/images/1150_jpg.rf.e2a4e07bcca74f8c0a698a4181be1d18.jpg  
 extracting: dataset/test/images/116260-IMG_9532_JPG_jpg.rf.ca2bf7ea5490c16f6

FileNotFoundError: Found no valid file for the classes labels. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [ ]:
import os

for root, dirs, files in os.walk("/content/dataset"):
    print(f"\n📁 Directory: {root}")
    for d in dirs:
        print(f" - 📂 Sub-folder: {d}")
    for f in files[:5]:  # show only first 5 files to keep it short
        print(f"   - 🖼️ File: {f}")



📁 Directory: /content/dataset
 - 📂 Sub-folder: train
 - 📂 Sub-folder: valid
 - 📂 Sub-folder: test
   - 🖼️ File: README.dataset.txt
   - 🖼️ File: README.roboflow.txt
   - 🖼️ File: data.yaml

📁 Directory: /content/dataset/train
 - 📂 Sub-folder: labels
 - 📂 Sub-folder: images

📁 Directory: /content/dataset/train/labels
   - 🖼️ File: 43j00lagmdcb1_jpg.rf.efd0fbda056f3ca2b3354a15b1cf78c4.txt
   - 🖼️ File: bought-my-car-last-year-and-previous-owner-had-put-body-v0-lwle0t06hlea1_webp.rf.8dbb71267b5ace354ff9ab983cf71d3d.txt
   - 🖼️ File: Scratch-Remover-Before-scaled_jpg.rf.0c35604c975096519c7269e2bdb4af66.txt
   - 🖼️ File: 164_scratch-repair-car-auto-11_jpg.rf.d89d4df37935476f74dc8884ffe9be2f.txt
   - 🖼️ File: image72_jpg.rf.4a42378c81df1ce9f29cd57e49ea82e5.txt

📁 Directory: /content/dataset/train/images
   - 🖼️ File: image62_jpeg.rf.8d327bfb5111b0e84dfd85b65a9535c1.jpg
   - 🖼️ File: images-102-_png_jpg.rf.b0f25e35320e11979ba13d9fb90a492a.jpg
   - 🖼️ File: vspuchivanie-kraski-na-kuzove_jpg.r

In [ ]:
# Step 1: Import libraries
import os
import torch
import torchvision
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report

# Step 2: Define YOLO-style custom Dataset
class YOLOClassificationDataset(Dataset):
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = sorted([
            f for f in os.listdir(images_dir)
            if f.endswith(('.jpg', '.jpeg', '.png'))
        ])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.images_dir, img_name)
        label_path = os.path.join(self.labels_dir, os.path.splitext(img_name)[0] + '.txt')

        # Load image
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        # Load first class_id from label file (assume single label per image)
        with open(label_path, 'r') as f:
            class_id = int(f.readline().split()[0])

        return image, class_id

# Step 3: Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Step 4: Set paths for train and validation
train_images = '/content/dataset/train/images'
train_labels = '/content/dataset/train/labels'
val_images = '/content/dataset/valid/images'
val_labels = '/content/dataset/valid/labels'

# Step 5: Create Datasets and Dataloaders
train_dataset = YOLOClassificationDataset(train_images, train_labels, transform=transform)
val_dataset = YOLOClassificationDataset(val_images, val_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Step 6: Prepare ResNet-50 Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(set(label for _, label in train_dataset))  # infer classes

model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Step 7: Train the model
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss:.4f}")

# Step 8: Evaluate the model
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("\nClassification Report:\n")
print(classification_report(all_labels, all_preds))


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 178MB/s]


Epoch [1/5], Loss: 18.8011
Epoch [2/5], Loss: 7.5342
Epoch [3/5], Loss: 2.1544
Epoch [4/5], Loss: 0.6296
Epoch [5/5], Loss: 0.2217

Classification Report:

              precision    recall  f1-score   support

           0       0.67      0.80      0.73         5
           1       0.64      0.78      0.70         9
           2       1.00      1.00      1.00        13
           3       0.57      0.80      0.67         5
           4       0.92      0.86      0.89        14
           5       0.82      1.00      0.90         9
           6       0.75      0.43      0.55        14

    accuracy                           0.80        69
   macro avg       0.77      0.81      0.78        69
weighted avg       0.81      0.80      0.79        69

